In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [11]:
dataset=dataset.drop("User ID",axis=1)

In [13]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [15]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [17]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [19]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [21]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [23]:
dependent=dataset[['Purchased']]

In [25]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [29]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
from sklearn.model_selection import GridSearchCV
param_grid={'criterion': ['gini', 'entropy', 'log_loss'],'splitter': ['best', 'random']}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
                    

grid.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [43]:
re=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [45]:
from sklearn. metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

In [47]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [49]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter{'criterion': 'gini', 'splitter': 'random'}: 0.8746214768227347


In [51]:
print("The confusion_matrix:\n",cm)

The confusion_matrix:
 [[72  7]
 [ 8 33]]


In [53]:
print ("The classification_report:\n",clf_report)

The classification_report:
               precision    recall  f1-score   support

           0       0.90      0.91      0.91        79
           1       0.82      0.80      0.81        41

    accuracy                           0.88       120
   macro avg       0.86      0.86      0.86       120
weighted avg       0.87      0.88      0.87       120



In [55]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.8581352269218895

In [57]:
table=pd.DataFrame.from_dict(re)

In [59]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011030,0.003967,0.016545,0.001880,gini,best,"{'criterion': 'gini', 'splitter': 'best'}",0.799537,0.840114,0.804584,0.911105,0.909115,0.852891,0.048772,3
1,0.007494,0.001829,0.015777,0.002568,gini,random,"{'criterion': 'gini', 'splitter': 'random'}",0.857143,0.858503,0.824293,0.874356,0.928571,0.868573,0.034135,1
2,0.005719,0.000664,0.015379,0.003579,entropy,best,"{'criterion': 'entropy', 'splitter': 'best'}",0.819142,0.840114,0.789949,0.892857,0.909115,0.850235,0.044692,4
3,0.005537,0.000783,0.017616,0.001427,entropy,random,"{'criterion': 'entropy', 'splitter': 'random'}",0.838326,0.840114,0.772092,0.768874,0.800053,0.803892,0.030822,6
4,0.006611,0.000934,0.016498,0.001849,log_loss,best,"{'criterion': 'log_loss', 'splitter': 'best'}",0.816207,0.858503,0.807155,0.839990,0.927778,0.849927,0.042894,5
5,0.005464,0.001583,0.015724,0.001148,log_loss,random,"{'criterion': 'log_loss', 'splitter': 'random'}",0.852966,0.858503,0.807155,0.892857,0.872761,0.856848,0.028415,2


In [61]:
age_input=int(input("Age:"))
bmi_input=int(input("BMI:"))
children_input=int(input("Children:"))
sex_male_input=int(input("Sex_male 0 or 1:"))
smoker_yes__input=int(input("Smoker_yes 0 or 1:"))

Age: 32
BMI: 43
Children: 2
Sex_male 0 or 1: 0
Smoker_yes 0 or 1: 1


In [63]:
Future_prediction=grid.predict([[age_input,bmi_input,children_input]])

In [65]:
Future_prediction

array([1], dtype=int64)